In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

In [2]:
class Model(tf.keras.Model):
  def __init__(self):
    super(Model, self).__init__()
    self.W = tfe.Variable(5., name='weight')
    self.B = tfe.Variable(10., name='bias')
  def predict(self, inputs):
    return inputs * self.W + self.B

In [3]:
# 최적화 되어야 할 loss 함수.
def loss(model, inputs, targets):
  error = model.predict(inputs) - targets
  return tf.reduce_mean(tf.square(error))

In [4]:
def grad(model, inputs, targets):
  with tfe.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, [model.W, model.B])

In [5]:
#  3 * x + 2 주위의 여러 점들을 포함하는 간단한 데이터셋입니다.
NUM_EXAMPLES = 2000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

In [6]:
# 다음을 정의합니다:
# 1. 모델.
# 2. 모델 파라미터에 대한 loss 함수의 미분.
# 3. 미분 결과에 따라 변수를 갱신할 방법.
model = Model()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

# 학습 루프
for i in range(300):
  grads = grad(model, training_inputs, training_outputs)
  optimizer.apply_gradients(zip(grads, [model.W, model.B]),
                            global_step=tf.train.get_or_create_global_step())
  if i % 20 == 0:
    print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

Initial loss: 68.764
Loss at step 000: 66.087
Loss at step 020: 30.067
Loss at step 040: 13.972
Loss at step 060: 6.779
Loss at step 080: 3.565
Loss at step 100: 2.128
Loss at step 120: 1.486
Loss at step 140: 1.199
Loss at step 160: 1.071
Loss at step 180: 1.013
Loss at step 200: 0.988
Loss at step 220: 0.976
Loss at step 240: 0.971
Loss at step 260: 0.969
Loss at step 280: 0.968
Final loss: 0.967
W = 3.0146803855895996, B = 2.0153846740722656
